In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn

from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import train_test_split

usar_gpu = torch.cuda.is_available()
# usar_gpu = False
if usar_gpu:
    print("CUDA disponível. Processamento em GPU!")
else:
    print("CUDA NÃO disponível. Processamento em CPU!")


CUDA disponível. Processamento em GPU!


In [2]:
class NeuralNet(nn.Module):
    def __init__(self) -> None:
        super(NeuralNet, self).__init__()

        self.seq1 = nn.Sequential(nn.Linear(5, 10), nn.Sigmoid())
        self.seq2 = nn.Sequential(nn.Linear(10, 50), nn.ReLU(inplace=True))
        self.seq21 = nn.Sequential(nn.Linear(50, 50), nn.ReLU(inplace=True))
        self.seq3 = nn.Sequential(nn.Linear(50, 10), nn.ReLU(inplace=True))
        self.seq4 = nn.Sequential(nn.Linear(10, 2), nn.Sigmoid())

    def forward(self, x):
        o = self.seq1(x)
        o = self.seq2(o)
        o = self.seq21(o)
        o = self.seq3(o)
        o = self.seq4(o)

        return o


In [3]:
class MFPTDataset(Dataset):
    def __init__(self, xs, ys) -> None:
        super().__init__()
        self.features = xs
        self.labels = ys

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index) :
        return self.features[index], self.labels[index]

data = np.load("MFPT_Features.npz")

x, y = data["x_features"], data["y_labels"]
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=4)

batch_size = 25
train_loader = DataLoader(MFPTDataset(xtrain, ytrain), batch_size, shuffle=True)
test_loader = DataLoader(MFPTDataset(xtest, ytest), batch_size, shuffle=True)

print("train shapes:", xtrain.shape, ytrain.shape)
print("test shapes: ", xtest.shape, ytest.shape)


train shapes: (1019, 5) (1019,)
test shapes:  (437, 5) (437,)


In [4]:
model = NeuralNet()
print(model)

if usar_gpu:
    torch.cuda.init()
    model.cuda()
    cudnn.benchmark = True



NeuralNet(
  (seq1): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): Sigmoid()
  )
  (seq2): Sequential(
    (0): Linear(in_features=10, out_features=50, bias=True)
    (1): ReLU(inplace=True)
  )
  (seq21): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): ReLU(inplace=True)
  )
  (seq3): Sequential(
    (0): Linear(in_features=50, out_features=10, bias=True)
    (1): ReLU(inplace=True)
  )
  (seq4): Sequential(
    (0): Linear(in_features=10, out_features=2, bias=True)
    (1): Sigmoid()
  )
)


In [5]:

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [6]:
n_epochs = 500

model.train()
for e in range(n_epochs):
    loss_total = 0

    for batch_idx, (x, y) in enumerate(train_loader):
        if usar_gpu:
            x, y = x.float().cuda(), y.cuda()
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        loss_total += loss

    loss_total = loss_total / len(ytrain)
    print(f"Epoca {e+1}/{n_epochs} \t Loss de Treinamento: {loss}")


Epoca 1/500 	 Loss de Treinamento: 0.7006053328514099
Epoca 2/500 	 Loss de Treinamento: 0.695509135723114
Epoca 3/500 	 Loss de Treinamento: 0.6992497444152832
Epoca 4/500 	 Loss de Treinamento: 0.6930257678031921
Epoca 5/500 	 Loss de Treinamento: 0.6924751400947571
Epoca 6/500 	 Loss de Treinamento: 0.68632572889328
Epoca 7/500 	 Loss de Treinamento: 0.6974618434906006
Epoca 8/500 	 Loss de Treinamento: 0.6721261143684387
Epoca 9/500 	 Loss de Treinamento: 0.6741512417793274
Epoca 10/500 	 Loss de Treinamento: 0.6829085350036621
Epoca 11/500 	 Loss de Treinamento: 0.6650465726852417
Epoca 12/500 	 Loss de Treinamento: 0.6818575263023376
Epoca 13/500 	 Loss de Treinamento: 0.6169257164001465
Epoca 14/500 	 Loss de Treinamento: 0.6349934935569763
Epoca 15/500 	 Loss de Treinamento: 0.6057677865028381
Epoca 16/500 	 Loss de Treinamento: 0.6291674375534058
Epoca 17/500 	 Loss de Treinamento: 0.5975169539451599
Epoca 18/500 	 Loss de Treinamento: 0.6556391716003418
Epoca 19/500 	 Loss de

In [7]:
model.eval()

loss_test = 0

score = 0

with torch.no_grad():
    for (x, y) in test_loader:
        if usar_gpu:
            x, y = x.float().cuda(), y.cuda()

        eout = model(x)
        loss = criterion(eout, y)
        loss_test += loss

        _, predicted = torch.max(eout.data, 1)

        for label, pred in zip(y, predicted):
            if label == pred:
                score += 1
        # print(y, predicted)

print(f'acc: {score/len(ytest)}')


acc: 0.9702517162471396
